In [21]:
import pandas as pd
import numpy as np

from pykrx import stock

pd.options.display.float_format = '{:.5f}'.format  # 지수표현 없이 출력
# pd.reset_option('display.float_format')          # 되돌리기

In [102]:
# 20100101 ~ 20210831 KOSPI 지수
df = stock.get_index_ohlcv_by_date("20100101", "20210831", '1001') 

In [104]:
# 날짜가 index로 되어있어서 column화
df = df.reset_index()

In [106]:
# 필요 컬럼만 남기기(날짜, 종가)
df = df[['날짜','종가']]

In [ ]:
# 월별 마지막 날짜(last_day) 컬럼 만들기
df['yyyymm'] = df.날짜.astype(str).apply(lambda x: x.replace("-","")[:6])
df['day'] = df.날짜.astype(str).apply(lambda x: x.replace("-","")[6:]).astype(int)
temp = df.groupby("yyyymm").day.max()
temp = pd.DataFrame(temp)
temp.columns = ['last_day']
df = pd.merge(df,temp,on='yyyymm')

In [ ]:
# 월별 마지막 일자 데이터만 남기기
df = df[df.day==df.last_day]
df.reset_index(drop=True, inplace=True)

In [ ]:
df['기간수익률'] = 0.0

for idx, data in df.iterrows() :
    if idx < 12:
        continue
    else:
        df.loc[idx,'기간수익률'] = (data['종가'] / df.loc[idx-12,'종가'] - 1)

In [ ]:
# 캡스톤 프로젝트에서 사용할 2011년부터 데이터만 남기기
df = df.iloc[12:,:]
df.reset_index(drop=True, inplace=True)
KOSPI_RETURNS = df[['yyyymm','종가','기간수익률']]
KOSPI_RETURNS.columns = ['yyyymm','close_price','one_y_returns']